# Moment tensor inversion in 3D for Western offset well
### Includes elastic model calibration test using perf. # 14
### Refer to chapter 4 for the 2D case and workflow

### Attributes list

0: stage \
1: perf number within a stage \
2: channel number corresponding to the shot apex \
3: channel number \
4: shot nominal MD (in m) \
5: channel MD (in m) \
6: shot x (in m) \
7: shot y (in m) \
8: shot z (in m) \
9: channel x (in m) \
10: channel y (in m) \
11: channel z (in m) \
12: rotated shot x (in m) \
13: rotated shot y (in m) \
14: rotated shot z (in m) \
15: rotated channel x (in m) \
16: rotated channel y (in m) \
17: rotated channel z (in m) \
18: channel dip (in radians) \
19: offset (in m) \
20: offsetx (in m) \
21: offsety (in m) \
22: offsetz (in m) \
23: flag to indicate active/dead trace \
24: empty

## Part I: model calibration
Objective: find the optimal velocities to flatten the P-, SH-, and SV-arrivals in field and synthetic data
Note that $V_S0$, and anisotropy have been already scanned and calibrated, thus field and synthetic data are expxected to agree kinematically.

In [ ]:
import sys

pwd = !echo ${PWD}
sys.path.append(pwd[0]+"/../../../code/local/bin")

In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy import ndimage
from scipy import optimize
import math
import copy
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from matplotlib import rcParams
from matplotlib.ticker import FormatStrFormatter


import seppy
import os

sep = seppy.sep()

rcParams['font.size'] = 8
rcParams['font.family'] = 'sans-serif'

datapath=pwd[0]+"/../dat/"
figpath=pwd[0]+"/../fig/"

In [ ]:
# utility functions

def plotseis(d, p=0.9, valmin=1, valmax=-1, tmin=0, tmax=0.35, xmin=-600, xmax=600, grid=True, colorbar=False, curve=np.array([]), linestyle='--'):
    """Plot 2D seismic image"""
 
    vmin=np.min(d) / p
    vmax=p*np.max(d)
    
    if p<0:
        vmax=-p*np.amax(d)
        vmin=-vmax
        
    if valmin<valmax:
        vmin=valmin
        vmax=valmax
        
    nx=d.shape[0]
                
    fig=plt.figure(figsize=(6, 4),dpi=100)
    plt.imshow(np.transpose(d),interpolation='sinc',aspect="auto",extent=[xmin,xmax,tmax,tmin],cmap='Greys',vmin=vmin,vmax=vmax)
    plt.xlabel('Offsetx (m)')
    plt.ylabel('Time (sec)')
    if grid==True:
        plt.grid(color='r', linestyle='-', linewidth=0.2)
    if colorbar==True:
        plt.colorbar()
    if curve.size>0:
        plt.plot(np.linspace(xmin,xmax,nx),curve,linestyle)
    plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
    
    plt.show()
    
def traveltime(v=1500, tshift=0.02, xmin=-600, xmax=600, nx=1201, y=300):
    """Compute travel time curve with constant velocity and fixed time shift and offsety"""
    
    tt = np.sqrt((np.linspace(xmin,xmax,nx))**2 + y**2)/v + tshift
    return tt

def lmo(d,xmin=-600,xmax=600,y=300,v=1500,tshift=0,dt=0.002):
    """Apply linear moveout to seismic data"""
    
    nx=d.shape[0]
    
    tt = traveltime(v,0,xmin,xmax,nx,y)
    lmo = np.copy(d)
        
    for ix in range(nx):
        lmo[ix,:] = ndimage.shift(d[ix,:],(-tt[ix]+tshift)/dt)
    
    return lmo
    
def envelop(d):
    """Compute the envelop via Hilbert transform along the fast axis"""
    
    return np.absolute(signal.hilbert(d,axis=-1))

def lmo_scan(d,xmin=-600,xmax=600,y=300,vmin=1500,vmax=5000,nv=101,tshift=0.1,dt=0.002,twidth=0.04):
    """Perform lmo scan with varying velocities and compute the maximum stack in a given time window"""
    
    scan = np.zeros((nv,3))
    dv = (vmax-vmin)/(nv-1.0)
    
    itmin = int(tshift/dt)
    itmax = itmin + int(twidth/dt)
    
    for iv in range(nv):
        v = vmin + dv*iv
        d_lmo = lmo(d,xmin,xmax,y,v,tshift,dt)
        scan[iv,0] = v
        scan[iv,1] = np.amax(np.abs(np.sum(d_lmo[:,itmin:itmax],axis=0)))
        scan[iv,2] = tshift + dt*np.argmax(np.abs(np.sum(d_lmo[:,itmin:itmax],axis=0)))
        
    return scan

def wavelet(nt=101, dt=0.002, t0=0, sig1=0.003, sig2=0.02):
    """Build a time function that is the derivative of the generalized Hubbert's pimple"""
    
    it0=int(nt/2)
    t=(np.linspace(0,nt-1,nt)-it0)*dt
    a=(sig2/sig1)**(-sig2/(sig1+sig2)) + (sig2/sig1)**(sig1/(sig1+sig2))
    a=a*a
    w=-a*(-np.exp((t0-t)/sig1)/sig1 + np.exp((t-t0)/sig2)/sig2)/(np.exp((t0-t)/sig1) + np.exp((t-t0)/sig2))**3
    
    return w

In [ ]:
# read data, Green's functions, mask etc...

# first stage 2
# last stage 12
# nb of stages 11
# there are 41 perfs

axes, data = sep.read_file(datapath+"ch6_data.H.bp")
pdata = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch6_data.HH")
pattr = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch6_mute.H")
pmute = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch6_dxGnx.H.bp")
pdxGnx = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch6_dyGny.H.bp")
pdyGny = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch6_dzGnz.H.bp")
pdzGnz = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch6_dxGny.H.bp")
pdxGny = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch6_dxGnz.H.bp")
pdxGnz = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch6_dyGnz.H.bp")
pdyGnz = data.reshape(axes.n,order='F').T


pdata *=pmute 
pdxGnx *=pmute 
pdyGny *=pmute 
pdzGnz *=pmute 
pdxGny *=pmute 
pdxGnz *=pmute 
pdyGnz *=pmute 

dt=axes.d[0]
dx=axes.d[1]
nt=axes.n[0]
ntr=axes.n[1]
ns=axes.n[2]

# apply constant time median filtering to the data to remove CMN
for s in range(ns):
    for it in range(pdata.shape[2]):
        med = np.median(pdata[s,:,it])
        pdata[s,:,it] -= med
pdata *=pmute

# select a single shot for Model calibration

sp=14
pdata1=pdata[sp]
pmute1=pmute[sp]
pattr1=pattr[sp]

pdata1=pdata1*pmute1

# choose the GFs to use in Model calibration
psyn = pdyGny[sp]*pmute1 # for P and SH
psyn2 = pdzGnz[sp]*pmute1 # for SV

In [ ]:
sp=30
plotseis(pdata[sp])
# plotseis(pdxGnx[sp])
# plotseis(pdyGny[sp])
# plotseis(pdzGnz[sp])
# plotseis(pdxGny[sp])
# plotseis(pdxGnz[sp])
# plotseis(pdyGnz[sp])

In [ ]:
# Figure of some shot examples

sp1=14
sp2=16
sp3=23
sp4=30
p=0.9
vmax=p*np.amax(pdata[sp1])

extent=[-600,600,0.35,0]

fig=plt.figure(figsize=(6.66, 6),dpi=300)

ax = plt.subplot(2,2,1)
plt.imshow(np.transpose(pdata[sp1]),interpolation='sinc',aspect="auto",extent=extent,cmap='Greys',vmin=-vmax,vmax=vmax)
plt.ylabel('Time (s)')
plt.gca().set_xticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'a) Shot #%d' %sp1,loc='left')
plt.annotate("P", xy=[-400,0.14],xytext=[-550,0.12],color='k', arrowprops=dict(facecolor='white', width=1, headlength=5, headwidth=4))
plt.annotate("SH", xy=[-400,0.22],xytext=[-550,0.19],color='k', arrowprops=dict(facecolor='white', width=1, headlength=5, headwidth=4))
plt.annotate("SV", xy=[-380,0.3],xytext=[-530,0.29],color='k', arrowprops=dict(facecolor='white', width=1, headlength=5, headwidth=4))

ax = plt.subplot(2,2,2)
plt.imshow(np.transpose(pdata[sp2]),interpolation='sinc',aspect="auto",extent=extent,cmap='Greys',vmin=-vmax,vmax=vmax)
plt.gca().set_xticklabels([])
plt.gca().set_yticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'b) Shot #%d' %sp2,loc='left')
plt.annotate(r"SH$^*$", xy=[-200,0.18],xytext=[-380,0.17],color='k', arrowprops=dict(facecolor='white', width=1, headlength=5, headwidth=4))

ax = plt.subplot(2,2,3)
plt.imshow(np.transpose(pdata[sp3]),interpolation='sinc',aspect="auto",extent=extent,cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'c) Shot #%d' %sp3,loc='left')
plt.annotate(r"SH$^*$", xy=[-100,0.2],xytext=[0,0.25],color='k', arrowprops=dict(facecolor='white', width=1, headlength=5, headwidth=4))

ax = plt.subplot(2,2,4)
plt.imshow(np.transpose(pdata[sp4]),interpolation='sinc',aspect="auto",extent=extent,cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.gca().set_yticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'd) Shot #%d' %sp4,loc='left')
plt.annotate(r"SH$^*$", xy=[-100,0.17],xytext=[0,0.2],color='k', arrowprops=dict(facecolor='white', width=1, headlength=5, headwidth=4))
plt.annotate("MS", xy=[180,0.33],xytext=[0,0.3],color='k', arrowprops=dict(facecolor='white', width=1, headlength=5, headwidth=4))

plt.tight_layout()
# plt.tight_layout(pad=0., w_pad=-2., h_pad=0.)

plt.savefig(figpath+'das.png',bbox_inches='tight',format='png')

In [ ]:
# GFs

sp=14
p=0.9
vmax=p*np.amax(pdxGnx[sp])

extent=[-600,600,0.35,0]

fig=plt.figure(figsize=(6.66, 9),dpi=300)

ax = plt.subplot(3,2,1)
plt.imshow(np.transpose(pdxGnx[sp]),interpolation='sinc',aspect="auto",extent=extent,cmap='Greys',vmin=-vmax,vmax=vmax)
plt.ylabel('Time (s)')
plt.gca().set_xticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'a) $M_{11}$',loc='left')

ax = plt.subplot(3,2,2)
plt.imshow(np.transpose(pdyGny[sp]),interpolation='sinc',aspect="auto",extent=extent,cmap='Greys',vmin=-vmax,vmax=vmax)
plt.gca().set_xticklabels([])
plt.gca().set_yticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'b) $M_{22}$',loc='left')

ax = plt.subplot(3,2,3)
plt.imshow(np.transpose(pdzGnz[sp]),interpolation='sinc',aspect="auto",extent=extent,cmap='Greys',vmin=-vmax,vmax=vmax)
plt.ylabel('Time (s)')
plt.gca().set_xticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'c) $M_{33}$',loc='left')

ax = plt.subplot(3,2,4)
plt.imshow(np.transpose(pdxGny[sp]),interpolation='sinc',aspect="auto",extent=extent,cmap='Greys',vmin=-vmax,vmax=vmax)
plt.gca().set_xticklabels([])
plt.gca().set_yticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'd) $M_{12}$',loc='left')

ax = plt.subplot(3,2,5)
plt.imshow(np.transpose(pdxGnz[sp]),interpolation='sinc',aspect="auto",extent=extent,cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'e) $M_{13}$',loc='left')

ax = plt.subplot(3,2,6)
plt.imshow(np.transpose(pdyGnz[sp]),interpolation='sinc',aspect="auto",extent=extent,cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.gca().set_yticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'f) $M_{23}$',loc='left')

plt.tight_layout()
# plt.tight_layout(pad=0., w_pad=-2., h_pad=0.)

plt.savefig(figpath+'GF.png',bbox_inches='tight',format='png')

In [ ]:
# apply lmo

offsetx_min = -600
offsetx_max = 600
offsety = pattr1[int(ntr/2),16] - pattr1[int(ntr/2),13] # offsety at offsetx=0 
vel = 2570 # velocity
tshift = 0.1 # time shift for lmo

data_lmo = lmo(pdata1,offsetx_min,offsetx_max,offsety,vel,tshift,dt)
syn_lmo = lmo(psyn,offsetx_min,offsetx_max,offsety,vel,tshift,dt)
syn2_lmo = lmo(psyn2,offsetx_min,offsetx_max,offsety,vel,tshift,dt)

# data_lmo = lmo(envelop(pdata1),offsetx_min,offsetx_max,offsety,vel,tshift,dt)
# syn_lmo = lmo(envelop(psyn),offsetx_min,offsetx_max,offsety,vel,tshift,dt)
# syn2_lmo = lmo(envelop(psyn2),offsetx_min,offsetx_max,offsety,vel,tshift,dt)

In [ ]:
plotseis(data_lmo,p=0.9)
plotseis(syn_lmo,p=0.9)
plotseis(syn2_lmo,p=0.9)

In [ ]:
# perform lmo scan using negative offsets

data_scan = lmo_scan(np.abs(pdata1[200:601,:]),xmin=-400,xmax=0,y=offsety,vmin=2400,vmax=5000,nv=261,tshift=0.11,dt=0.002,twidth=0.05)
syn_scan = lmo_scan(np.abs(psyn[200:601,:]),xmin=-400,xmax=0,y=offsety,vmin=2400,vmax=5000,nv=261,tshift=0.11,dt=0.002,twidth=0.05)

data_scan_sv = lmo_scan(np.abs(pdata1[200:601,:]),xmin=-400,xmax=0,y=offsety,vmin=1700,vmax=2100,nv=41,tshift=0.11,dt=0.002,twidth=0.08)
syn_scan_sv = lmo_scan(np.abs(psyn2[200:601,:]),xmin=-400,xmax=0,y=offsety,vmin=1700,vmax=2100,nv=41,tshift=0.11,dt=0.002,twidth=0.08)

In [ ]:
# smooth the velocity scan curves by median and mean filtering then normalize
# smooth the time shift scan by median filtering

kernel_size = 7
mean_filter=np.ones(kernel_size)/kernel_size

data_vel_filt = signal.medfilt(data_scan[:,1],kernel_size=kernel_size)
data_vel_filt = np.convolve(data_vel_filt, mean_filter, mode = 'same')
data_vel_filt = np.convolve(data_vel_filt, mean_filter, mode = 'same')
data_vel_filt = data_vel_filt/np.max(data_vel_filt)
data_tshift_filt = signal.medfilt(data_scan[:,2],kernel_size=kernel_size)

syn_vel_filt = signal.medfilt(syn_scan[:,1],kernel_size=kernel_size)
syn_vel_filt = np.convolve(syn_vel_filt, mean_filter, mode = 'same')
syn_vel_filt = np.convolve(syn_vel_filt, mean_filter, mode = 'same')
syn_vel_filt = syn_vel_filt/np.max(syn_vel_filt)
syn_tshift_filt = signal.medfilt(syn_scan[:,2],kernel_size=kernel_size)

data_vel_filt_sv = signal.medfilt(data_scan_sv[:,1],kernel_size=kernel_size)
data_vel_filt_sv = np.convolve(data_vel_filt_sv, mean_filter, mode = 'same')
data_vel_filt_sv = np.convolve(data_vel_filt_sv, mean_filter, mode = 'same')
data_vel_filt_sv = data_vel_filt_sv/np.max(data_vel_filt_sv)
data_tshift_filt_sv = signal.medfilt(data_scan_sv[:,2],kernel_size=kernel_size)

syn_vel_filt_sv = signal.medfilt(syn_scan_sv[:,1],kernel_size=kernel_size)
syn_vel_filt_sv = np.convolve(syn_vel_filt_sv, mean_filter, mode = 'same')
syn_vel_filt_sv = np.convolve(syn_vel_filt_sv, mean_filter, mode = 'same')
syn_vel_filt_sv = syn_vel_filt_sv/np.max(syn_vel_filt_sv)
syn_tshift_filt_sv = signal.medfilt(syn_scan_sv[:,2],kernel_size=kernel_size)

In [ ]:
%matplotlib notebook
# plot velocity scans and pick the three horizontal velocities V_p, V_sh, and V_sv
plt.plot(data_scan[:,0, ], data_vel_filt, label="data")
plt.plot(data_scan[:,0, ], syn_vel_filt, label="syn")
plt.plot(data_scan_sv[:,0, ], data_vel_filt_sv, label="data")
plt.plot(data_scan_sv[:,0, ], syn_vel_filt_sv, label="syn")
plt.legend()
plt.xlabel("Velocity (m/s)")
plt.ylabel("Maximum stack amplitude")
plt.show()

In [ ]:
# plot time shift scans for QC
plt.plot(data_scan[:,0, ], data_tshift_filt, label="data")
plt.plot(data_scan[:,0, ], syn_tshift_filt, label="syn")
plt.plot(data_scan_sv[:,0, ], data_tshift_filt_sv, label="data")
plt.plot(data_scan_sv[:,0, ], syn_tshift_filt_sv, label="syn")
plt.legend()
plt.xlabel("Velocity (m/s)")
plt.ylabel("Maximum stack amplitude")
plt.show()

### Observations/Conclusions

Based on the velocity scan, estimated effective horizontal velocities form field data are:
$$V_P = 4360 \,m/s \quad ; \quad V_{SH} = 2570 \,m/s \quad ; \quad V_{SV} = 1930 \,m/s$$

For synthetic data using Gny,y for P and SH and Gnz,z for SV:
$$V_P = 4280 \,m/s \quad ; \quad V_{SH} = 2550 \,m/s \quad ; \quad V_{SV} = unclear \,m/s$$

These are more like phase than group velocities. For synthetic data, the results may slightly change depending on which GF is being used.

The $V_{SH}$ is matching well between field and synthetic data whereas time shifts agree well for P-arrival. The synthetic SH seems to arrive 4 ms later than field data. Due to the uncertainty of the LMO scan method, I will not use it to calibrate synthetic data. I will use instead the GF's as is since the model has been previously calibrated.

Note that the group velocity can be estimated using the envelop of the signal, however the noise in the data makes the estimation less robust.

## Part II: MT inversion - first pass

In [ ]:
%matplotlib inline

In [ ]:
ngf=6

pGF=np.zeros((ns,ngf,ntr, nt))
pGF[:,0,:,:]=pdxGnx
pGF[:,1,:,:]=pdyGny
pGF[:,2,:,:]=pdzGnz
pGF[:,3,:,:]=pdxGny
pGF[:,4,:,:]=pdxGnz
pGF[:,5,:,:]=pdyGnz

In [ ]:
# separate P-, SH-, and SV_arrivals in field data and GFs
offsetx_min = -600
offsetx_max = 600
noffset=offsetx_max-offsetx_min + 1
v_obs=np.array([4360,2570,1930])
v_syn=np.array([4360,2570,1930])
tshift_obs = np.array([0.11, 0.11, 0.12]) # time shifts for lmo
tshift_syn = tshift_obs + 0.0
twidth = np.array([0.05, 0.05, 0.08]) # window widths to separate arrivals
itwidth_beg=np.array([0,int(twidth[0]/dt),int(twidth[0]/dt)+int(twidth[1]/dt)], dtype=np.int32)
itwidth_end=np.array([int(twidth[0]/dt),int(twidth[0]/dt)+int(twidth[1]/dt),int(twidth[0]/dt)+int(twidth[1]/dt)+int(twidth[2]/dt)], dtype=np.int32)
itwidth=itwidth_end-itwidth_beg
itwidth_cumul=np.sum(itwidth)

# field data
d=[]
for s in range(ns):
    offsety = pattr[s,int(ntr/2),16] - pattr[s,int(ntr/2),13] # offsety at offsetx=0 
    temp1=np.zeros((noffset,itwidth_cumul))
    for v in range(3):
        temp2 = lmo(pdata[s,:noffset,:],offsetx_min,offsetx_max,offsety,v_obs[v],tshift_obs[v],dt)
        temp1[:,itwidth_beg[v]:itwidth_end[v]] = temp2[:,int(tshift_obs[v]/dt):int(tshift_obs[v]/dt)+itwidth[v]]
    d.append(np.copy(temp1))
    
# GFs
gf=[]
for s in range(ns):
    offsety = pattr[s,int(ntr/2),16] - pattr[s,int(ntr/2),13] # offsety at offsetx=0 
    temp1=np.zeros((6,noffset,itwidth_cumul))
    for g in range(ngf):
        for v in range(3):
            temp2 = lmo(pGF[s,g,:noffset,:],offsetx_min,offsetx_max,offsety,v_syn[v],tshift_syn[v],dt)
            temp1[g,:,itwidth_beg[v]:itwidth_end[v]] = temp2[:,int(tshift_syn[v]/dt):int(tshift_syn[v]/dt)+itwidth[v]]
    gf.append(np.copy(temp1))

In [ ]:
# combine into numpy array to save to disk
dtemp=np.zeros((ns,d[0].shape[0],d[0].shape[1]))
for s in range(ns):
    dtemp[s]=d[s]

# save to seplib
sep.write_file("dtemp.H", np.transpose(dtemp), ds=np.array([1,1,1]), os=np.array([0,0,0]), dpath=pwd[0]+"/")

# apply FK filter
!${PWD}/../../../code/local/bin/FK_FILTER.x < dtemp.H kmin=-0.008 kmax=0.008 taper=0.01 > dtemp.H.fk datapath=./

# read back
axes, dtemp = sep.read_file("dtemp.H.fk")
dtemp = dtemp.reshape(axes.n,order='F').T

# remove files from disk
!rm -f dtemp*

# replace the data list
for s in range(ns):
    d[s]=dtemp[s]

In [ ]:
# build a mask to be used in the MT inversion
mask = np.zeros(d[0].shape)
# mask[:,:itwidth_end[0]+2]=1 # keep full P-arrival
mask[:,:itwidth_end[1]+2]=1 # keep full P- and SH-arrivals
mask[:501,:]=1 # keep SV- and SH-arrival for negative offsets
mask[:300,:]=0 # mask offsets<-300 m
mask[901:,:]=0 # mask offsets>300 m
# mask[501:700,:]=0 # mask -100<offsets<100 m
mask[:,itwidth_end[1]:]=0 # remove SV arrival

In [ ]:
# Figure of some shot examples

sp1=14
sp2=16
sp3=23
sp4=30
p=0.8
vmax=p*np.amax(pdata[sp1])

extent=[-600,600,0.18,0]

fig=plt.figure(figsize=(6.66, 6),dpi=300)

ax = plt.subplot(2,2,1)
plt.imshow(np.transpose(d[sp1]),interpolation='sinc',aspect="auto",extent=extent,cmap='Greys',vmin=-vmax,vmax=vmax)
plt.ylabel('Time (s)')
plt.gca().set_xticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'a) Shot #%d' %sp1,loc='left')
plt.annotate("P", xy=[400,0.03],xytext=[500,0.03],color='k', arrowprops=dict(facecolor='white', width=1, headlength=5, headwidth=4))
plt.annotate("SH", xy=[400,0.07],xytext=[500,0.07],color='k', arrowprops=dict(facecolor='white', width=1, headlength=5, headwidth=4))
plt.annotate("SV", xy=[400,0.16],xytext=[500,0.16],color='k', arrowprops=dict(facecolor='white', width=1, headlength=5, headwidth=4))

ax = plt.subplot(2,2,2)
plt.imshow(np.transpose(d[sp2]),interpolation='sinc',aspect="auto",extent=extent,cmap='Greys',vmin=-vmax,vmax=vmax)
plt.gca().set_xticklabels([])
plt.gca().set_yticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'b) Shot #%d' %sp2,loc='left')

ax = plt.subplot(2,2,3)
plt.imshow(np.transpose(d[sp3]),interpolation='sinc',aspect="auto",extent=extent,cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'c) Shot #%d' %sp3,loc='left')

ax = plt.subplot(2,2,4)
plt.imshow(np.transpose(d[sp4]),interpolation='sinc',aspect="auto",extent=extent,cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.gca().set_yticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'd) Shot #%d' %sp4,loc='left')

plt.tight_layout()
# plt.tight_layout(pad=0., w_pad=-2., h_pad=0.)

plt.savefig(figpath+'das_processed.png',bbox_inches='tight',format='png')

In [ ]:
# GFs

sp=14
p=0.8
vmax=p*np.amax(pdxGnx[sp])

extent=[-600,600,0.18,0]

fig=plt.figure(figsize=(6.66, 9),dpi=300)

ax = plt.subplot(3,2,1)
plt.imshow(np.transpose(gf[sp][0]),interpolation='sinc',aspect="auto",extent=extent,cmap='Greys',vmin=-vmax,vmax=vmax)
plt.ylabel('Time (s)')
plt.gca().set_xticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'a) $M_{11}$',loc='left')

ax = plt.subplot(3,2,2)
plt.imshow(np.transpose(gf[sp][1]),interpolation='sinc',aspect="auto",extent=extent,cmap='Greys',vmin=-vmax,vmax=vmax)
plt.gca().set_xticklabels([])
plt.gca().set_yticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'b) $M_{22}$',loc='left')

ax = plt.subplot(3,2,3)
plt.imshow(np.transpose(gf[sp][2]),interpolation='sinc',aspect="auto",extent=extent,cmap='Greys',vmin=-vmax,vmax=vmax)
plt.ylabel('Time (s)')
plt.gca().set_xticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'c) $M_{33}$',loc='left')

ax = plt.subplot(3,2,4)
plt.imshow(np.transpose(gf[sp][3]),interpolation='sinc',aspect="auto",extent=extent,cmap='Greys',vmin=-vmax,vmax=vmax)
plt.gca().set_xticklabels([])
plt.gca().set_yticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'd) $M_{12}$',loc='left')

ax = plt.subplot(3,2,5)
plt.imshow(np.transpose(gf[sp][4]),interpolation='sinc',aspect="auto",extent=extent,cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'e) $M_{13}$',loc='left')

ax = plt.subplot(3,2,6)
plt.imshow(np.transpose(gf[sp][5]),interpolation='sinc',aspect="auto",extent=extent,cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.gca().set_yticklabels([])
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'f) $M_{23}$',loc='left')

plt.tight_layout()
# plt.tight_layout(pad=0., w_pad=-2., h_pad=0.)

plt.savefig(figpath+'GF_processed.png',bbox_inches='tight',format='png')

In [ ]:
sp=14
plotseis(d[sp],p=0.9, tmin=0, tmax=np.sum(twidth), xmin=offsetx_min, xmax=offsetx_max)
for i in range(ngf):
    plotseis(gf[sp][i],p=0.9, tmin=0, tmax=np.sum(twidth), xmin=offsetx_min, xmax=offsetx_max)
plotseis(mask,p=1, tmin=0, tmax=np.sum(twidth), xmin=offsetx_min, xmax=offsetx_max)

In [ ]:
# for every shot
#    scan sigma1, sigma2>=sigma1, origin time
#    convolve GFs with the time function
#    build matrix G and invert the MT
#    find the smallest normalized functional
#    deduce the optimal origin time, sigma1, sigma2

nt0=d[0].shape[1]
origin_t=np.zeros(ns)
sigma1=np.zeros(ns)
sigma2=np.zeros(ns)
nfunc=999*np.ones(ns)
mt=np.zeros((ns,6))

# time shift boundaries in sec
otmin=-0.012
otmax=0.004
otnum=9

# sigma boundaries is sec
sigmin=dt/2
signum1=4
signum2=7

# full functional
nfunc_all=np.zeros((ns,otnum*signum1*signum2))

# shots to invert
spmin=0
spmax=41

# container for synthetic data
dsyn=copy.deepcopy(d)

In [ ]:
offmin=300 # use only limited offsets in the MT inversion
offmax=901

mask0 = mask[offmin:offmax,:]

# loop over shots
for s in range(spmin,spmax):
    
    print("Processing shot %d" %s)
    
    # data norm
    d0 = (d[s][offmin:offmax,:]*mask0).flatten()

    dnorm=np.linalg.norm(d0)
    
    # make copies of GFs
    gf_w = np.copy(gf[s])
        
    # full functional for the given shot
    func=np.zeros(0)
    
    # loop over sigmas
    for sig1 in np.linspace(sigmin,signum1*sigmin,signum1):
        for sig2 in np.linspace(sigmin,signum2*sigmin,signum2):
            
            if (sig2>=sig1):

                # loop over time origin
                for shift in np.linspace(otmin,otmax,otnum):
            
                    # build the shifted time function
                    w = wavelet(nt0, dt, shift, sig1, sig2)
    
                    # convolve GF with the time functions
                    for g in range(ngf):
                        for itr in range(noffset):
                            gf_w[g,itr,:] = dt*np.convolve(gf[s][g,itr,:],w,mode='same')

                    # form the matrix G
                    G0=np.vstack((gf_w[0,:,:].flatten(),gf_w[1,:,:].flatten()))
                    G=np.vstack(((gf_w[0,offmin:offmax,:]*mask0).flatten(),(gf_w[1,offmin:offmax,:]*mask0).flatten()))
                    
                    for g in range(2,ngf):
                        G0=np.vstack((G0,gf_w[g,:,:].flatten()))
                        G=np.vstack((G,(gf_w[g,offmin:offmax,:]*mask0).flatten()))                  
                    
                    G0=np.transpose(G0)
                    G=np.transpose(G)
                    
                    # Invert the MT using the pseudo inverse of G
                    m=np.matmul(np.linalg.pinv(G),d0)

                    # Compute predicted data
                    d_hat0=np.matmul(G0,m)
                    d_hat=np.matmul(G,m)

                    # Compute the normalized functional
                    f=np.linalg.norm(d_hat-d0)/dnorm
                    func=np.append(func,f)
                    
                    # check if the normalized residual is the smallest so far
                    if (f<nfunc[s]):
                        nfunc[s]=f
                        mt[s,:]=m
                        origin_t[s]=shift
                        sigma1[s]=sig1
                        sigma2[s]=sig2
                        dsyn[s]=np.reshape(d_hat0,(noffset,nt0))
            else:
                # loop over time origin
                for shift in np.linspace(otmin,otmax,otnum):
                    func=np.append(func,9999)
                    
    nfunc_all[s] = func

In [ ]:
# save the outputs
np.save(datapath+'origin_t',origin_t)
np.save(datapath+'sigma1',sigma1)
np.save(datapath+'sigma2',sigma2)
np.save(datapath+'mt',mt)
np.save(datapath+'nfunc',nfunc)
np.save(datapath+'nfunc_all',nfunc_all)
for s in range(spmin,spmax):
    np.save(datapath+'syn'+str(s),dsyn[s])

In [ ]:
# load the outputs
origin_t=np.load(datapath+'origin_t.npy')
sigma1=np.load(datapath+'sigma1.npy')
sigma2=np.load(datapath+'sigma2.npy')
mt=np.load(datapath+'mt.npy')
nfunc=np.load(datapath+'nfunc.npy')
nfunc_all=np.load(datapath+'nfunc_all.npy')
for s in range(spmin,spmax):
    dsyn[s]=np.load(datapath+'syn'+str(s)+'.npy')

In [ ]:
sp=14
ff=nfunc_all[sp].reshape((signum1,signum2,otnum))
vmin=np.min(ff)
#vmax=np.max(ff)
vmax=1.1
plt.figure(figsize=(12,8))
for i in range(otnum-3):
    plt.subplot(2,3,i+1)
    plt.imshow(ff[:,:,i],cmap='coolwarm',extent=[1000*sigmin,1000*signum2*sigmin,1000*signum1*sigmin,1000*sigmin],vmin=vmin,vmax=vmax)
#     plt.colorbar(fraction=0.04)
    ot=otmin+i*(otmax-otmin)/(otnum-1)
    plt.title(r"$t_0$=%.0f msec" %(1000*ot))
    if (abs(origin_t[sp]-ot)<0.00001):
        plt.scatter(1000*sigma2[sp],1000*sigma1[sp],c='w',s=100)        
    plt.gca().invert_yaxis()
    if i>5:
        plt.xlabel(r'$\sigma_2$ (msec)')
    else:
        plt.gca().set_xticklabels([])
    if i in [0, 3, 6]:
        plt.ylabel(r'$\sigma_1$ (msec)')
    else:
        plt.gca().set_yticklabels([])
    
#     plt.gca().set(aspect='equal')
#     plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
#     plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.1f'))

plt.tight_layout(pad=0., w_pad=0., h_pad=0.)
# plt.savefig(figpath+'func_global_s.png',bbox_inches='tight',format='png')

In [ ]:
plt.figure(figsize=(6.66,4),dpi=300)

plt.subplot(2,2,1)
bins=1000*np.linspace(sigmin,4*sigmin,4)-0.5
plt.hist(1000*sigma1,bins=bins)
plt.xlabel(r'Optimal $\sigma_1$ (ms)')
plt.ylabel("Shot count",fontsize=12)
plt.xticks([1,2,3,4])
plt.gca().tick_params(axis='both', which='major', labelsize=6)

plt.subplot(2,2,2)
bins=1000*np.linspace(sigmin,9*sigmin,9)-0.5
plt.hist(1000*sigma2,bins=bins)
plt.xlabel(r'Optimal $\sigma_2$ (ms)')
plt.xticks()
plt.gca().tick_params(axis='both', which='major', labelsize=6)

plt.subplot(2,2,4)
bins=np.linspace(-12,4,9)-1
plt.hist(1000*origin_t,bins=bins)
plt.xlabel(r'Optimal time shift (ms)')
plt.ylabel("Shot count",fontsize=12)
plt.gca().tick_params(axis='both', which='major', labelsize=6)

plt.tight_layout()

plt.savefig(figpath+'hist_s1s2t.png',bbox_inches='tight',format='png')

print("Median sigma1 = %f sec" %np.median(sigma1))
print("Median sigma2 = %f sec" %np.median(sigma2))
print("Median t0 = %f sec" %np.median(origin_t))

In [ ]:
plt.figure(figsize=(12,10))
plt.subplot(2,2,1)
plt.hist(1000*sigma1,bins=np.linspace(1,5,5),align='left')
plt.xlabel(r'Optimal $\sigma_1$ (ms)',fontsize=12)
plt.ylabel("Shot count",fontsize=12)
plt.xticks([1,2,3,4,5])

plt.subplot(2,2,2)
n,bins,patches=plt.hist(1000*sigma2,bins=np.linspace(1,10,10),align='left')
plt.xlabel(r'Optimal $\sigma_2$ (ms)',fontsize=12)
plt.xticks(np.arange(1,signum2+1))

plt.subplot(2,2,3)
plt.hist(1000*origin_t,bins=1000*np.linspace(otmin,otmax,otnum),align='left')
plt.xlabel(r'Optimal time shift (ms)',fontsize=12)
plt.ylabel("Shot count",fontsize=12)

plt.subplot(2,2,4)
n,bins,patches=plt.hist(nfunc)
plt.xlabel(r'Normalized residual norm',fontsize=12)

#plt.savefig(figpath+'hist_s1s2t.png',bbox_inches='tight',format='png')

print("Median sigma1 = %f sec" %np.median(sigma1))
print("Median sigma2 = %f sec" %np.median(sigma2))
print("Median t0 = %f sec" %np.median(origin_t))
print("Median nfunc = %f" %np.median(nfunc))

In [ ]:
plt.figure(figsize=(14,10))
plt.subplot(3,3,1)
plt.hist(1000*mt[:,0],bins=10)
plt.xlabel(r"$M_{11}$ (scaled N.m)",fontsize=12)
plt.ylabel("Shot count",fontsize=12)
plt.ylim([0, 15])

plt.subplot(3,3,2)
plt.hist(1000*mt[:,1],bins=10)
plt.xlabel(r"$M_{22}$ (scaled N.m)",fontsize=12)
# plt.ylabel("Shot count",fontsize=12)
plt.gca().set_yticklabels([])
plt.ylim([0, 15])

plt.subplot(3,3,3)
plt.hist(1000*mt[:,2],bins=10)
plt.xlabel(r"$M_{33}$ (scaled N.m)",fontsize=12)
# plt.ylabel("Shot count",fontsize=12)
plt.gca().set_yticklabels([])
plt.ylim([0, 15])

plt.subplot(3,3,4)
plt.hist(1000*mt[:,3],bins=10)
plt.xlabel(r"$M_{12}$ (scaled N.m)",fontsize=12)
plt.ylabel("Shot count",fontsize=12)
plt.ylim([0, 15])

plt.subplot(3,3,5)
plt.hist(1000*mt[:,4],bins=10)
plt.xlabel(r"$M_{13}$ (scaled N.m)",fontsize=12)
# plt.ylabel("Shot count",fontsize=12)
plt.gca().set_yticklabels([])
plt.ylim([0, 15])

plt.subplot(3,3,6)
plt.hist(1000*mt[:,5],bins=10)
plt.xlabel(r"$M_{23}$ (scaled N.m)",fontsize=12)
# plt.ylabel("Shot count",fontsize=12)
plt.gca().set_yticklabels([])
plt.ylim([0, 15])

plt.tight_layout()

In [ ]:
sp=10
plotseis(d[sp],p=-0.7, valmin=-4, valmax=4, tmin=0, tmax=np.sum(twidth), xmin=offsetx_min, xmax=offsetx_max)
plotseis(dsyn[sp],p=-0.7, valmin=-4, valmax=4, tmin=0, tmax=np.sum(twidth), xmin=offsetx_min, xmax=offsetx_max)

## Part III: MT inversion - second pass

In [ ]:
# build another mask to be used in the MT inversion
mask2 = np.zeros(d[0].shape)
mask2[:,:itwidth_end[0]+2]=1 # keep full P-arrival
mask2[:801,:itwidth_end[1]+2]=1 # keep part of P- and SH-arrivals, offset <= 200 m
mask2[:200,:]=0 # mask offsets<-400 m
mask2[1001:,:]=0 # mask offsets>400 m
mask2[:,itwidth_end[1]:]=0 # remove SV arrival

In [ ]:
plotseis(mask2,p=1, tmin=0, tmax=np.sum(twidth), xmin=offsetx_min, xmax=offsetx_max)

In [ ]:
# Trace alignment and selection based on max correlation with synthetics

# for every shot
#    compute correlation with synthetic
#    shift to maximum correlation
#    zero traces with NCC < percentile

d2=copy.deepcopy(d)
gf2=copy.deepcopy(gf)

tshifts=np.zeros((ns,d[0].shape[0]))
ncctr=np.zeros((ns,d[0].shape[0]))

spmin=0
spmax=41

lagmin=-0
lagmax=0

percentile=20

for s in range(spmin,spmax):
            
    for itr in range(ntr):
        
        # 1d correlation
        corr=np.correlate(mask2[itr,:]*d[s][itr,:],mask2[itr,:]*dsyn[s][itr,:],mode='full')
        
        # maximum correlation
        ind_max=np.argmax(corr)
        
        # deduce time shift
        ts=ind_max-(d[0].shape[1]-1)
        tshifts[s,itr]=-ts
        
        # set to zero if outside the lag range
#         if ts<lagmin or ts>lagmax:
#             ts=0
            
        # clip the lag range
        if ts<lagmin:
            ts=lagmin
        if ts>lagmax:
            ts=lagmax
            
        # apply the shifts
        d2[s][itr,:]=ndimage.shift(d[s][itr,:],-ts)
        
        # calculate the normalized cross-correlation coefficients NCC
        d_norm=np.linalg.norm(d2[s][itr,:])
        dsyn_norm=np.linalg.norm(dsyn[s][itr,:])
        if (d_norm * dsyn_norm) > 1e-8:                    
            ncctr[s,itr] = np.dot(d2[s][itr,:],dsyn[s][itr,:])/(d_norm * dsyn_norm)
        
    # calculate the median of NCC
    perc=np.percentile(ncctr[s,:],percentile)
    
    # zero out traces with NCC < median
    tozero = np.argwhere(ncctr[s,:]<max(0,perc))
    d2[s][tozero,:]=0
    for c in range(6):
        gf2[s][c,tozero,:]=0

In [ ]:
sp=14
plotseis(d2[sp],p=-0.7, tmin=0, tmax=np.sum(twidth), xmin=offsetx_min, xmax=offsetx_max)
plotseis(gf2[sp][1,:,:],p=-0.7, tmin=0, tmax=np.sum(twidth), xmin=offsetx_min, xmax=offsetx_max)

In [ ]:
offmin=200 # use only limited offsets in the MT inversion
offmax=1001

dsyn2=copy.deepcopy(d)

mask0 = mask2[offmin:offmax,:]

nfunc2=np.zeros((ns))
mt2=np.zeros((ns,6))
cond2=np.zeros((ns))

# clip the shift values
origin_t_clip=np.clip(origin_t,-0.004,0.004)

# select median values
sig1 = np.median(sigma1)
sig2 = np.median(sigma2)
shift = np.median(origin_t_clip)

# loop over shots
for s in range(ns):
    
    # get the time wavelet parameters
    sig1 = sigma1[s]
    sig2 = sigma2[s]
#     shift = origin_t_clip[s]
    shift = origin_t[s]
    
    # build the time function
    w = wavelet(nt0, dt, shift, sig1, sig2)
        
    # data norm
    d0 = (d2[s][offmin:offmax,:]*mask0).flatten()

    dnorm=np.linalg.norm(d0)
    
    # make copies of GFs
    gf0_w = np.copy(gf[s])
    gf_w = np.copy(gf2[s])

    # convolve GF with the time functions
    for g in range(ngf):
        for itr in range(noffset):
            gf0_w[g,itr,:] = dt*np.convolve(gf[s][g,itr,:],w,mode='same')
            gf_w[g,itr,:] = dt*np.convolve(gf2[s][g,itr,:],w,mode='same')

    # form the matrix G
    G0=np.vstack((gf0_w[0,:,:].flatten(),gf0_w[1,:,:].flatten()))
    G=np.vstack(((gf_w[0,offmin:offmax,:]*mask0).flatten(),(gf_w[1,offmin:offmax,:]*mask0).flatten()))

    for g in range(2,ngf):
        G0=np.vstack((G0,gf0_w[g,:,:].flatten()))
        G=np.vstack((G,(gf_w[g,offmin:offmax,:]*mask0).flatten()))                  

    G0=np.transpose(G0)
    G=np.transpose(G)

    # Invert the MT using the pseudo inverse of G
    m=np.matmul(np.linalg.pinv(G),d0)

    # Compute predicted data
    d_hat0=np.matmul(G0,m)
    d_hat=np.matmul(G,m)

    # Compute the normalized functional
    f=np.linalg.norm(d_hat-d0)/dnorm
    
    nfunc2[s]=f
    mt2[s,:]=m[:]
    cond2[s]=np.linalg.cond(G)
    dsyn2[s]=np.reshape(d_hat0,(noffset,nt0))

In [ ]:
# save the outputs
np.save(datapath+'mt2',mt2)

In [ ]:
# load the outputs
mt2=np.load(datapath+'mt2.npy')

In [ ]:
fig = plt.figure(figsize=(6.66, 5),dpi=300)

plt.subplot(2,3,1)
plt.hist(-10000*mt2[:,0],bins=10)
plt.xlabel(r"$M_{11}$ (scaled N.m)")
plt.ylabel("Shot count")
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.ylim([0, 15])

plt.subplot(2,3,2)
plt.hist(-10000*mt2[:,1],bins=10)
plt.xlabel(r"$M_{22}$ (scaled N.m)")
plt.gca().set_yticklabels([])
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.ylim([0, 15])

plt.subplot(2,3,3)
plt.hist(-10000*mt2[:,2],bins=10)
plt.xlabel(r"$M_{33}$ (scaled N.m)")
plt.gca().set_yticklabels([])
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.ylim([0, 15])

plt.subplot(2,3,4)
plt.hist(-10000*mt2[:,3],bins=10)
plt.xlabel(r"$M_{12}$ (scaled N.m)")
plt.ylabel("Shot count")
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.ylim([0, 15])

plt.subplot(2,3,5)
plt.hist(-10000*mt2[:,4],bins=10)
plt.xlabel(r"$M_{13}$ (scaled N.m)")
plt.gca().set_yticklabels([])
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.ylim([0, 15])

plt.subplot(2,3,6)
plt.hist(-10000*mt2[:,5],bins=10)
plt.xlabel(r"$M_{23}$ (scaled N.m)")
plt.gca().set_yticklabels([])
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.ylim([0, 15])

plt.tight_layout()

plt.savefig(figpath+'mt_histogram.png',bbox_inches='tight',format='png')

In [ ]:
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.hist(cond2,bins=10)
plt.xlabel("Condition number")
plt.ylabel("Shot count")

plt.subplot(1,2,2)
n,bins,patches=plt.hist(nfunc2)
plt.xlabel(r'Normalized residual norm',fontsize=12)

print("Smallest condition number = %f" %np.min(cond2))
print("Largest condition number = %f" %np.max(cond2))
print("Median nfunc = %f" %np.median(nfunc2))

In [ ]:
sp=14
plotseis(d[sp], valmin=-4, valmax=4, tmin=0, tmax=np.sum(twidth), xmin=offsetx_min, xmax=offsetx_max)
plotseis(dsyn[sp],valmin=-4, valmax=4, tmin=0, tmax=np.sum(twidth), xmin=offsetx_min, xmax=offsetx_max)
plotseis(dsyn2[sp],valmin=-4, valmax=4, tmin=0, tmax=np.sum(twidth), xmin=offsetx_min, xmax=offsetx_max)

In [ ]:
# compute full time functions and synthetic data by convolvng GFs with the time function and weight by MT
pfull_time_func=np.zeros((ns,6,nt))
psyn_all = np.copy(pdxGnx)
for s in range(ns):
    w0 = wavelet(nt=nt, dt=dt, t0=origin_t[s], sig1=sigma1[s], sig2=sigma2[s])
    w = ndimage.shift(w0,-(nt-1)/2.0+0.02/dt) # shift so the first sample corresponds to -20 ms (as the GF's)
    for c in range(6):
        pfull_time_func[s,c,:]=mt2[s,c]*w
    for itr in range(ntr):
        temp = mt2[s,0]*pGF[s,0,itr,:] + mt2[s,1]*pGF[s,1,itr,:] + mt2[s,2]*pGF[s,2,itr,:] + mt2[s,3]*pGF[s,3,itr,:] + mt2[s,4]*pGF[s,4,itr,:] + mt2[s,5]*pGF[s,5,itr,:]
        psyn_all[s,itr,:]=dt*np.convolve(temp,w0,mode='same')

In [ ]:
np.savetxt(datapath+"ch6_scaler_MTs.txt",mt2, fmt='%.9f')
sep.write_file(datapath+"ch6_all_time_func.H", np.transpose(pfull_time_func), ds=np.array([dt,1,1]), os=np.array([0,0,0]), dpath=datapath)
sep.write_file(datapath+"ch6_synthetic.H", np.transpose(psyn_all), ds=np.array([dt,1,1]), os=np.array([0,-600,0]), dpath=datapath)

In [ ]:
sp=14
plotseis(pdata[sp])
plotseis(psyn_all[sp])

## Part IV: constrained MT inversion, decomposition, and plotting

In [ ]:
home = !echo ${HOME}
sys.path.append(home[0]+"/.local/lib/python3.9/site-packages")

In [ ]:
import MTfit
from MTfit.plot import MTplot
import mpltern

In [ ]:
def getStiffness(mod):
    """Build the 21 elements of the upper triangular part of the Voigt stiffness matrix given the VTI 
    elastic parameters"""
    
    vp=mod[0]
    vs=mod[1]
    rho=mod[2]
    delta=mod[3]
    epsilon=mod[4]
    gamma=mod[5]
    
    c33 = rho*vp*vp
    c11 = (1+2*epsilon)*c33
    c55 = rho*vs*vs
    c66 = (1+2*gamma)*c55
    c13 = np.sqrt(2*c33*(c33-c55)*delta + np.square(c33-c55)) - c55                   
    mu = c55
    lam = c33 - 2*mu
    
    C=np.zeros((21))
    
    C[0]=c11
    C[1]=c11-2*c66
    C[2]=c13
    C[6]=c11
    C[7]=c13
    C[11]=c33
    C[15]=c55
    C[18]=c55
    C[20]=c66
    
    return C

def MT6_to_MT33(mt, forward_mode=True):
    """Convert MT from 6-vector to 3x3 tensor an vice versa"""
    
    if forward_mode:
        mt33=np.zeros((3,3))
        
        mt33[0,0]=mt[0]
        mt33[1,1]=mt[1]
        mt33[2,2]=mt[2]
        mt33[0,1]=mt[3]
        mt33[0,2]=mt[4]
        mt33[1,2]=mt[5]
        mt33[1,0]=mt33[0,1]
        mt33[2,0]=mt33[0,2]
        mt33[2,1]=mt33[1,2]
        
        return mt33
    else:
        mt6 = np.zeros((6))
        
        mt6[0]=mt[0,0]
        mt6[1]=mt[1,1]
        mt6[2]=mt[2,2]
        mt6[3]=mt[0,1]
        mt6[4]=mt[0,2]
        mt6[5]=mt[1,2]
        
        return mt6

def rotate(M,theta,phi):
    """Rotate a 3x3 tensor"""
    
    c = math.cos(math.radians(phi))
    s = math.sin(math.radians(phi))
    R1=np.array(([c,-s,0],[s,c,0],[0,0,1]))

    c = math.cos(math.radians(theta))
    s = math.sin(math.radians(theta))
    R2=np.array(([c,0,-s],[0,1,0],[s,0,c]))
    
    R = np.matmul(R1,R2)
    Rt = np.transpose(R)
        
    M_rot = np.matmul(M,R)
    M_rot = np.matmul(Rt,M_rot)
    
    return M_rot

def rotationMatrix(theta,phi):
    """Build the 6 x 6 rotation matrix acting on a flattened symmetric 3 x 3 tensor
    ordered as xx, yy, zz, xy, xz, yz"""
    
    rot = np.zeros((6,6))
    for i in range(6):
        M_in = np.zeros((3,3))
        if i<3:
            M_in[i,i] = 1
        elif i==3:
            M_in[0,1] = 1
            M_in[1,0] = 1
        elif i==4:
            M_in[0,2] = 1
            M_in[2,0] = 1
        else:
            M_in[1,2] = 1
            M_in[2,1] = 1
        
        M_out=rotate(M_in, theta, phi)
        rot[:,i] = MT6_to_MT33(M_out, forward_mode=False)
    
    return rot

def rotationY(angle):
    """Build rotation matrix around y-axis ; the angle is in degrees"""

    c = math.cos(math.radians(angle))
    s = math.sin(math.radians(angle))
    R=np.array(([c,0,-s],[0,1,0],[s,0,c]))

    return R

def rotationZ(angle):
    """Build rotation matrix around z-axis ; the angle is in degrees"""

    c = math.cos(math.radians(angle))
    s = math.sin(math.radians(angle))
    R=np.array(([c,-s,0],[s,c,0],[0,0,1]))

    return R

def rotateYZ(xyz,theta,phi):
    """Rotate coordinates around y- and z-axes ; angles are in degrees"""

    Ry = rotationY(theta)
    Rz = rotationZ(phi)
    v = np.matmul(Rz,xyz)
    v = np.matmul(Ry,v)

    return v

def scaleMT6(mt):
    """Scale the last 3 components by sqrt(2) to be used for plotting in MTfit"""
    
    mtScaled = np.copy(mt)
    mtScaled[3] *= math.sqrt(2)
    mtScaled[4] *= math.sqrt(2)    
    mtScaled[5] *= math.sqrt(2)
    
    return mtScaled

def plotBiaxes(BA):
    """Plot the bi-axial decomposition of a moment (potency) tensor"""
                
    fig=plt.figure(figsize=(6, 4),dpi=100)
    
    plt.plot([-1,0],[0,1],'k')
    plt.plot([0,1],[1,0],'k')
    plt.plot([-1,0],[0,-1],'k')
    plt.plot([0,1],[-1,0],'k')
    plt.plot([-1,1],[0,0],'k')
    plt.plot([0,0],[-1,1],'k')
    plt.scatter(BA[:,0],BA[:,1])
    
    plt.axis('off')
    plt.annotate("+ISO",(-0.05,1.05),annotation_clip=False)
    plt.annotate("-ISO",(-0.05,-1.1),annotation_clip=False)
    plt.annotate("+TC",(-1.15,-0.05),annotation_clip=False)
    plt.annotate("-TC",(1.02,-0.05),annotation_clip=False)
    plt.annotate("DC",(0.01,0.02),annotation_clip=False)
        
    plt.show()

In [ ]:
# read VTI model and source coordinates (in the rotated system)
axes, data = sep.read_file(datapath+"ch6_model.H")
pmod = data.reshape(axes.n,order='F').T

sxyz=pattr[:,0,[0, 12, 13, 14]]
sxyz[:,1:] *= 0.001 # convert to km

Zo=axes.o[0]
Xo=axes.o[1]
Yo=axes.o[2]
Zd=axes.d[0]
Xd=axes.d[1]
Yd=axes.d[2]

In [ ]:
# Read "trajectories" of monitor, Western, and Eastern wells ; the trajectories are actually the nominal perforation locations

monitor=pd.read_excel("../../../input_data/auxiliary_data/PerforationInfo4SEP-v1.xlsx", sheet_name='Monitor Well')
east=pd.read_excel("../../../input_data/auxiliary_data/PerforationInfo4SEP-v1.xlsx", sheet_name='East Treat Well')
west=pd.read_excel("../../../input_data/auxiliary_data/PerforationInfo4SEP-v1.xlsx", sheet_name='West Treat Well')

# Calibrate the trajectories to the reference monitor one
monitor_traj=np.array([monitor["DX-Monitor"],monitor["DY-Monitor"],monitor["Z"]])
east_traj=np.array([east["DX-Monitor"],east["DY-Monitor"],east["Z"]])
west_traj=np.array([west["DX-Monitor"],west["DY-Monitor"],west["Z"]])

monitor_traj=np.flip(monitor_traj,axis=1)*0.3048 # flip the order and convert from ft to meters
east_traj=np.flip(east_traj,axis=1)*0.3048
west_traj=np.flip(west_traj,axis=1)*0.3048

monitor_traj[2,:] *= -1 # from negative to positive depth
east_traj[2,:] *= -1 
west_traj[2,:] *= -1

# calibration values estimated visually by comparison with reference (full) monitor trajectory
x_calib = 4
y_calib = 62
z_calib = 387.2

monitor_traj[0,:] += x_calib
east_traj[0,:] += x_calib 
west_traj[0,:] += x_calib
monitor_traj[1,:] += y_calib
east_traj[1,:] += y_calib 
west_traj[1,:] += y_calib
monitor_traj[2,:] += z_calib
east_traj[2,:] += z_calib 
west_traj[2,:] += z_calib

# rotate the coordinates
theta = -0.648114658
phi = 54.4

monitor_traj_rot = np.copy(monitor_traj)
east_traj_rot = np.copy(east_traj)
west_traj_rot = np.copy(west_traj)

for i in range(monitor_traj.shape[1]):
    monitor_traj_rot[:,i] = rotateYZ(monitor_traj[:,i],theta,phi)
    
for i in range(east_traj.shape[1]):
    east_traj_rot[:,i] = rotateYZ(east_traj[:,i],theta,phi)
    
for i in range(west_traj.shape[1]):
    west_traj_rot[:,i] = rotateYZ(west_traj[:,i],theta,phi)

In [ ]:
# calculate the well orientation (theta, phi) based on well trajectory
npts=west_traj.shape[1]
orientation=np.zeros((npts,2))
arr = np.transpose(west_traj_rot)
for s in range(1,npts-1):
    orientation[s,0] = math.atan( (arr[s+1,2]-arr[s-1,2]) / (math.sqrt(arr[s+1,0]**2 + arr[s+1,1]**2) - math.sqrt(arr[s-1,0]**2 + arr[s-1,1]**2)) )
    orientation[s,1] = math.atan( (arr[s+1,1] - arr[s-1,1]) / (arr[s+1,0] - arr[s-1,0]) )
s=0
orientation[s,0] = math.atan( (arr[s+1,2]-arr[s,2]) / (math.sqrt(arr[s+1,0]**2 + arr[s+1,1]**2) - math.sqrt(arr[s,0]**2 + arr[s,1]**2)) )
orientation[s,1] = math.atan( (arr[s+1,1] - arr[s,1]) / (arr[s+1,0] - arr[s,0]) )
s=npts-1
orientation[s,0] = math.atan( (arr[s,2]-arr[s-1,2]) / (math.sqrt(arr[s,0]**2 + arr[s,1]**2) - math.sqrt(arr[s-1,0]**2 + arr[s-1,1]**2)) )
orientation[s,1] = math.atan( (arr[s,1] - arr[s-1,1]) / (arr[s,0] - arr[s-1,0]) )

# find the well orientation at the shot locations
orientationSP=np.zeros((ns,2))
for s in range(ns):
    arr = np.transpose(west_traj_rot)-1000*sxyz[s,1:]
    arr = np.linalg.norm(arr,axis=1)
    index = np.argmin(arr)
    orientationSP[s,:] = orientation[index,:]

In [ ]:
# extract vp, vs, rho, delta, epsilon, gamma at source locations
pmod_src=np.zeros((6,ns))

for s in range(ns):
    
    ix = int((sxyz[s,1] - Xo)/Xd)
    iy = int((sxyz[s,2] - Yo)/Yd)
    iz = int((sxyz[s,3] - Zo)/Zd)
    wx = (sxyz[s,1] - ix*Xd - Xo)/Xd
    wy = (sxyz[s,2] - iy*Yd - Yo)/Yd
    wz = (sxyz[s,3] - iz*Zd - Zo)/Zd
    
    for comp in range(6):
        pmod_src[comp,s] = (1-wx)*(1-wy)*(1-wz)*pmod[comp,iy,ix,iz] \
                + (wx)*(1-wy)*(1-wz)*pmod[comp,iy,ix+1,iz] \
                + (1-wx)*(wy)*(1-wz)*pmod[comp,iy+1,ix,iz] \
                + (1-wx)*(1-wy)*(wz)*pmod[comp,iy,ix,iz+1] \
                + (wx)*(wy)*(1-wz)*pmod[comp,iy+1,ix+1,iz] \
                + (wx)*(1-wy)*(wz)*pmod[comp,iy,ix+1,iz+1] \
                + (1-wx)*(wy)*(wz)*pmod[comp,iy+1,ix,iz+1] \
                + (wx)*(wy)*(wz)*pmod[comp,iy+1,ix+1,iz+1]

In [ ]:
# deduce C33, C11, C55, C66, C13, lambda, mu
c33 = pmod_src[2]*pmod_src[0]*pmod_src[0]
c11 = (1+2*pmod_src[4])*c33
c55 = pmod_src[2]*pmod_src[1]*pmod_src[1]
c66 = (1+2*pmod_src[5])*c55
c13 = np.sqrt(2*c33*(c33-c55)*pmod_src[3] + np.square(c33-c55)) - c55                   
mu = c55
lam = c33 - 2*mu

In [ ]:
# run a constrained MT inversion to obey the source model proposed in Chapter 4
# The MT must be rotated in the direction of the well before using that source model

dsyn3=copy.deepcopy(d)
nfunc3=np.zeros((ns))
mt3=np.zeros((ns,6))
cond3=np.zeros((ns))

# build expansion matrix to transform the model m=[mxx,myy,mzz,myz] to [mxx,myy,mzz,0,0,myz]
E=np.zeros((6,4))
E[0,0]=1
E[1,1]=1
E[2,2]=1
E[5,3]=1

# loop over shots
for s in range(ns):
    
    # build the 6x6 rotation matrix
    R = rotationMatrix(math.degrees(-orientationSP[s,0]),math.degrees(-orientationSP[s,1]))
    
    # get the time wavelet parameters
    sig1 = sigma1[s]
    sig2 = sigma2[s]
    shift = origin_t[s]
    
    # build the time function
    w = wavelet(nt0, dt, shift, sig1, sig2)
        
    # data norm
    d0 = (d2[s][offmin:offmax,:]*mask0).flatten()

    dnorm=np.linalg.norm(d0)
    
    # make copies of GFs
    gf0_w = np.copy(gf[s])
    gf_w = np.copy(gf2[s])

    # convolve GF with the time functions
    for g in range(ngf):
        for itr in range(noffset):
            gf0_w[g,itr,:] = dt*np.convolve(gf[s][g,itr,:],w,mode='same')
            gf_w[g,itr,:] = dt*np.convolve(gf2[s][g,itr,:],w,mode='same')

    # form the matrix G
    G0=np.vstack((gf0_w[0,:,:].flatten(),gf0_w[1,:,:].flatten()))
    G=np.vstack(((gf_w[0,offmin:offmax,:]*mask0).flatten(),(gf_w[1,offmin:offmax,:]*mask0).flatten()))

    for g in range(2,ngf):
        G0=np.vstack((G0,gf0_w[g,:,:].flatten()))
        G=np.vstack((G,(gf_w[g,offmin:offmax,:]*mask0).flatten()))                  

    G0=np.transpose(G0)
    G=np.transpose(G)
    
    # form the full forward modeling matrix A=GRE
    RE=np.matmul(R,E)
    A=np.matmul(G,RE)
    A0=np.matmul(G0,RE)
    

    # Invert the MT using the pseudo inverse of A
    m=np.matmul(np.linalg.pinv(A),d0)

    # Compute predicted data
    d_hat0=np.matmul(A0,m)
    d_hat=np.matmul(A,m)

    # Compute the normalized functional
    f=np.linalg.norm(d_hat-d0)/dnorm
    
    nfunc3[s]=f
    mt3[s,:]=np.matmul(RE,m)
    cond3[s]=np.linalg.cond(A)
    dsyn3[s]=np.reshape(d_hat0,(noffset,nt0))

In [ ]:
# decompose the unconstrained MT inversion into CE, DF, and CO components according to the source model proposed in Chapter 4
# The MT must be rotated in the direction of the well before using that source model
# After MT rotation, the M12 and M13 components will be dropped implicitly

mt4=np.zeros((ns,6)) # mt2 rotated
cond4=np.zeros((3,ns))
# mt4 decomposed with phasing angles 0 (vertical), 45 (oblique), and 90 (horizontal) degrees
mt4_decomp_0=np.zeros((ns,3)) 
mt4_decomp_45=np.zeros((ns,3))
mt4_decomp_90=np.zeros((ns,3))

mt5_0 = np.zeros((ns,6))
mt5_45 = np.zeros((ns,6))
mt5_90 = np.zeros((ns,6))

# loop over shots
for s in range(ns):
    
    # build the 6x6 rotation matrix
    R = rotationMatrix(math.degrees(orientationSP[s,0]),math.degrees(orientationSP[s,1]))
    
    # rotate the MT into the well coord system
    mt4[s]=np.matmul(R,mt2[s])
    
    # drop M12 and M13, multiply by -1 because of the suspected data polarity flip
    d0 = -mt4[s,[0,1,2,5]]
    
    # build the matrices A0, A45, and A90
    A0=np.zeros((4,3))
    A45=np.zeros((4,3))
    A90=np.zeros((4,3))
    
    tce = np.array([ c11[s]-2*c66[s]+c13[s] , c11[s]+c13[s] , c33[s]+c13[s], 0 ])
    tdf0 = np.array([ 0, 0, 1, 0 ])
    tdf45 = np.array([ 0, 0.5, 0.5, -0.5 ])
    tdf90 = np.array([ 0, 1, 0, 0 ])
    tco0 = np.array([ c11[s]-c66[s] , c11[s]-c66[s] , c13[s] , 0 ])
    tco45 = np.array([ c11[s]-c66[s] , (c11[s]-c66[s]+c13[s])/2 , (c11[s]-c66[s]+c13[s])/2 , (c11[s]-c66[s]-c13[s])/2 ])
    tco90 = np.array([ c11[s]-c66[s] , c13[s], c11[s]-c66[s] , 0 ])
    
    A0[:,0] = tce/np.linalg.norm(tce)
    A0[:,1] = tdf0
    A0[:,2] = tco0/math.sqrt(np.sum(tco0**2)+tco0[3]**2)
    A0 = math.sqrt(2)*A0
    
    A45[:,0] = tce/np.linalg.norm(tce)
    A45[:,1] = tdf45
    A45[:,2] = tco45/math.sqrt(np.sum(tco45**2)+tco45[3]**2)
    A45 = math.sqrt(2)*A45
    
    A90[:,0] = tce/np.linalg.norm(tce)
    A90[:,1] = tdf90
    A90[:,2] = tco90/math.sqrt(np.sum(tco90**2)+tco90[3]**2)
    A90 = math.sqrt(2)*A90
    
    # Deduce the decompositions
#     mt4_decomp_0[s] = np.matmul(np.linalg.pinv(A0),d0)
#     mt4_decomp_45[s] = np.matmul(np.linalg.pinv(A45),d0)
#     mt4_decomp_90[s] = np.matmul(np.linalg.pinv(A90),d0)
    mt4_decomp_0[s], res  = optimize.nnls(A0, d0, maxiter=20)
    mt4_decomp_45[s], res = optimize.nnls(A45, d0, maxiter=20)
    mt4_decomp_90[s], res = optimize.nnls(A90, d0, maxiter=20)

    # condition numbers
    cond4[0,s]=np.linalg.cond(A0)
    cond4[1,s]=np.linalg.cond(A45)
    cond4[2,s]=np.linalg.cond(A90)
    
    # total MT = Mce + Mdf + Mco
    mt5_0[s,[0,1,2,5]] = np.matmul(A0, mt4_decomp_0[s])
    mt5_45[s,[0,1,2,5]] = np.matmul(A45, mt4_decomp_45[s])
    mt5_90[s,[0,1,2,5]] = np.matmul(A90, mt4_decomp_90[s])


In [ ]:
bins=np.linspace(0,1,11)-0.05

fig = plt.figure(figsize=(8, 6),dpi=300)

plt.subplot(3,1,1)
plt.hist((math.sqrt(2)*mt4_decomp_0[:,0]/np.linalg.norm(mt5_0,axis=1), math.sqrt(2)*mt4_decomp_0[:,1]/np.linalg.norm(mt5_0,axis=1), math.sqrt(2)*mt4_decomp_0[:,2]/np.linalg.norm(mt5_0,axis=1)), bins=bins, density=True, alpha=1, label=('CE','DF','CO'), color=('r','y','b'))
plt.gca().set_xticklabels([])
plt.ylabel("Normalized count")
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.legend()
plt.title(r'a) $\theta=0^o$', loc='left')

plt.subplot(3,1,2)
plt.hist((math.sqrt(2)*mt4_decomp_45[:,0]/np.linalg.norm(mt5_45,axis=1), math.sqrt(2)*mt4_decomp_45[:,1]/np.linalg.norm(mt5_45,axis=1), math.sqrt(2)*mt4_decomp_45[:,2]/np.linalg.norm(mt5_45,axis=1)), bins=bins, density=True, alpha=1, label=('CE','DF','CO'), color=('r','y','b'))
plt.gca().set_xticklabels([])
plt.ylabel("Normalized count")
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.legend()
plt.title(r'b) $\theta=45^o$', loc='left')

plt.subplot(3,1,3)
plt.hist((math.sqrt(2)*mt4_decomp_90[:,0]/np.linalg.norm(mt5_90,axis=1), math.sqrt(2)*mt4_decomp_90[:,1]/np.linalg.norm(mt5_90,axis=1), math.sqrt(2)*mt4_decomp_90[:,2]/np.linalg.norm(mt5_90,axis=1)), bins=bins, density=True, alpha=1, label=('CE','DF','CO'), color=('r','y','b'))
plt.xlabel(r"Normalized magnitude")
plt.ylabel("Normalized count")
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.legend()
plt.title(r'c) $\theta=90^o$', loc='left')


plt.tight_layout()
plt.savefig(figpath+'mt_histogram_decomposed.png',bbox_inches='tight',format='png')

In [ ]:
t0 = mt4_decomp_0[:,0]/np.sum(mt4_decomp_0,axis=1)
l0 = mt4_decomp_0[:,1]/np.sum(mt4_decomp_0,axis=1)
r0 = mt4_decomp_0[:,2]/np.sum(mt4_decomp_0,axis=1)

t45 = mt4_decomp_45[:,0]/np.sum(mt4_decomp_45,axis=1)
l45 = mt4_decomp_45[:,1]/np.sum(mt4_decomp_45,axis=1)
r45 = mt4_decomp_45[:,2]/np.sum(mt4_decomp_45,axis=1)

t90 = mt4_decomp_90[:,0]/np.sum(mt4_decomp_90,axis=1)
l90 = mt4_decomp_90[:,1]/np.sum(mt4_decomp_90,axis=1)
r90 = mt4_decomp_90[:,2]/np.sum(mt4_decomp_90,axis=1)

fig = plt.figure(figsize=(12, 6),dpi=300)

ax = fig.add_subplot(1, 3, 1, projection="ternary")
pc = ax.scatter(t0, l0, r0, c=10000*np.linalg.norm(mt5_0,axis=1), clip_on=False)

ax.set_tlabel('CE')
ax.set_llabel('DF')
ax.set_rlabel('CO')

# cax = ax.inset_axes([1.05, 0.1, 0.05, 0.9])
# colorbar = fig.colorbar(pc, cax=cax, pad=0.01, shrink=0.2)
# colorbar.set_label("Magnitude (scaled N.m)")

ax.set_title(r'a) $\theta=0^o$', loc='left')
plt.gca().tick_params(axis='both', which='major', labelsize=6)


ax = fig.add_subplot(1, 3, 2, projection="ternary")
pc = ax.scatter(t45, l45, r45, c=10000*np.linalg.norm(mt5_45,axis=1), clip_on=False)

ax.set_tlabel('CE')
ax.set_llabel('DF')
ax.set_rlabel('CO')

# cax = ax.inset_axes([1.05, 0.1, 0.05, 0.9])
# colorbar = fig.colorbar(pc, cax=cax, pad=0.01, shrink=0.2)
# colorbar.set_label("Magnitude (scaled N.m)")

ax.set_title(r'b) $\theta=45^o$', loc='left')
plt.gca().tick_params(axis='both', which='major', labelsize=6)


ax = fig.add_subplot(1, 3, 3, projection="ternary")
pc = ax.scatter(t90, l90, r90, c=10000*np.linalg.norm(mt5_90,axis=1), clip_on=False)

ax.set_tlabel('CE')
ax.set_llabel('DF')
ax.set_rlabel('CO')

cax = ax.inset_axes([1.05, 0.1, 0.05, 0.9])
colorbar = fig.colorbar(pc, cax=cax, pad=0.01, shrink=0.2)
colorbar.set_label("Magnitude (scaled N.m)")

ax.set_title(r'c) $\theta=90^o$', loc='left')
plt.gca().tick_params(axis='both', which='major', labelsize=6)

plt.tight_layout()
plt.savefig(figpath+'mt_ternary_decomposition.png',bbox_inches='tight',format='png')

In [ ]:
fig = plt.figure(figsize=(6.66, 5),dpi=300)

plt.subplot(2,3,1)
plt.hist(-10000*mt3[:,0],bins=10)
plt.xlabel(r"$M_{11}$ (scaled N.m)")
plt.ylabel("Shot count")
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.ylim([0, 15])

plt.subplot(2,3,2)
plt.hist(-10000*mt3[:,1],bins=10)
plt.xlabel(r"$M_{22}$ (scaled N.m)")
plt.gca().set_yticklabels([])
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.ylim([0, 15])

plt.subplot(2,3,3)
plt.hist(-10000*mt3[:,2],bins=10)
plt.xlabel(r"$M_{33}$ (scaled N.m)")
plt.gca().set_yticklabels([])
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.ylim([0, 15])

plt.subplot(2,3,4)
plt.hist(-10000*mt3[:,3],bins=10)
plt.xlabel(r"$M_{12}$ (scaled N.m)")
plt.ylabel("Shot count")
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.ylim([0, 15])

plt.subplot(2,3,5)
plt.hist(-10000*mt3[:,4],bins=10)
plt.xlabel(r"$M_{13}$ (scaled N.m)")
plt.gca().set_yticklabels([])
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.ylim([0, 15])

plt.subplot(2,3,6)
plt.hist(-10000*mt3[:,5],bins=10)
plt.xlabel(r"$M_{23}$ (scaled N.m)")
plt.gca().set_yticklabels([])
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.ylim([0, 15])

plt.tight_layout()

plt.savefig(figpath+'mt_histogram_constrained.png',bbox_inches='tight',format='png')

In [ ]:
for i in range(6):
    print("Median values for MT component %d for unconstrained and constrained inversion %f %f" %(i+1,-1e4*np.median(mt2[:,i]),-1e4*np.median(mt3[:,i])) )

In [ ]:
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.hist(cond3,bins=10)
plt.xlabel("Condition number")
plt.ylabel("Shot count")

plt.subplot(1,2,2)
n,bins,patches=plt.hist(nfunc3)
plt.xlabel(r'Normalized residual norm',fontsize=12)

print("Smallest condition number = %f" %np.min(cond3))
print("Largest condition number = %f" %np.max(cond3))
print("Median nfunc = %f" %np.median(nfunc3))

In [ ]:
# compute synthetic data by convolvng GFs with the time function and weight by MT
psyn_all2 = np.copy(pdxGnx)
for s in range(ns):
    w0 = wavelet(nt=nt, dt=dt, t0=origin_t[s], sig1=sigma1[s], sig2=sigma2[s])
    w = ndimage.shift(w0,-(nt-1)/2.0+0.02/dt) # shift so the first sample corresponds to -20 ms (as the GF's)
    for itr in range(ntr):
        temp = mt3[s,0]*pGF[s,0,itr,:] + mt3[s,1]*pGF[s,1,itr,:] + mt3[s,2]*pGF[s,2,itr,:] + mt3[s,3]*pGF[s,3,itr,:] + mt3[s,4]*pGF[s,4,itr,:] + mt3[s,5]*pGF[s,5,itr,:]
        psyn_all2[s,itr,:]=dt*np.convolve(temp,w0,mode='same')

In [ ]:
# extract vp, vs, rho, delta, epsilon, gamma at source locations
mod=np.zeros((6,ns))

for s in range(ns):
    
    ix = int((sxyz[s,1] - Xo)/Xd)
    iy = int((sxyz[s,2] - Yo)/Yd)
    iz = int((sxyz[s,3] - Zo)/Zd)
    wx = (sxyz[s,1] - ix*Xd - Xo)/Xd
    wy = (sxyz[s,2] - iy*Yd - Yo)/Yd
    wz = (sxyz[s,3] - iz*Zd - Zo)/Zd
    
    for c in range(6):
        mod[c,s] = (1-wx)*(1-wy)*(1-wz)*pmod[c,iy,ix,iz] \
                +  (wx)*(1-wy)*(1-wz)*pmod[c,iy,ix+1,iz] \
                +  (1-wx)*(wy)*(1-wz)*pmod[c,iy+1,ix,iz] \
                +  (1-wx)*(1-wy)*(wz)*pmod[c,iy,ix,iz+1] \
                +  (wx)*(wy)*(1-wz)*pmod[c,iy+1,ix+1,iz] \
                +  (wx)*(1-wy)*(wz)*pmod[c,iy,ix+1,iz+1] \
                +  (1-wx)*(wy)*(wz)*pmod[c,iy+1,ix,iz+1] \
                +  (wx)*(wy)*(wz)*pmod[c,iy+1,ix+1,iz+1]

# construct the source tensor (potency tensor) and the bi-axes decomposition (Chapman and leaney, 2011)
# from the MTs
# bi-axes decomposition, 3-tuple: 3x2 containing the bi-axes (defining fault plane and displacement-discontinuity
# direction), explosion value, area displacement value
D = np.copy(mt2) # potency tensor
D2= np.copy(mt3)
BA = np.zeros((ns,2))
MT_scaled=np.copy(mt2)
MT_scaled2=np.copy(mt3)
for s in range(ns):
    C = getStiffness(mod[:,s])
    MT_scaled[s,:] = scaleMT6(mt2[s])
    MT_scaled2[s,:] = scaleMT6(mt3[s])
    Dtemp = MTfit.convert.moment_tensor_conversion.MT6c_D6(MT_scaled[s,:], C.tolist())
    D[s,:] = Dtemp.squeeze()
    
    Dtemp = MTfit.convert.moment_tensor_conversion.MT6c_D6(MT_scaled2[s,:], C.tolist())
    D2[s,:] = Dtemp.squeeze()
    
    BAtemp=MTfit.convert.moment_tensor_conversion.MT6_biaxes(MT_scaled[s,:], C.tolist())
    phi = np.dot(BAtemp[0][:,0], BAtemp[0][:,1])/(np.linalg.norm(BAtemp[0][:,0])*np.linalg.norm(BAtemp[0][:,1])) # angle between bi-axes
    phi = math.acos(phi)
    chi = math.pi / 2. - phi
    N = math.sin(chi) * math.sqrt(2./(1 + (math.sin(chi))**2 ))
    BA[s,1] = BAtemp[1] / (abs(BAtemp[1]) + 2*BAtemp[2]) # normalized explosion part
    BA[s,0] = -N*(1 - abs(BA[s,1])) # normalized displacement discontinuity part

In [ ]:
# tag the last or second to last perf in each stage
select1=np.array([4,8,11,14,19,23,26,29,32,36,40])
select2=select1 - 1
tags=np.empty((ns),dtype='object')
tags[:]='k'
tags[select1]='r'
tags[select2]='b'

In [ ]:
MTplot(np.transpose(-MT_scaled2),plot_type='hudson',projection='uv',color=tags,save_file='',save_dpi=300)
MTplot(np.transpose(-D2),plot_type='hudson',projection='uv',color=tags,save_file='',save_dpi=300)

In [ ]:
MTplot(np.transpose(-MT_scaled),plot_type='hudson',projection='uv',color=tags,save_file=figpath+'Hudson_iso_fullMT.png',save_dpi=300)
MTplot(np.transpose(-MT_scaled2),plot_type='hudson',projection='uv',color=tags,save_file=figpath+'Hudson_iso_constrainedMT.png',save_dpi=300)
MTplot(np.transpose(-D),plot_type='hudson',projection='uv',color=tags,save_file=figpath+'Hudson_aniso_fullMT.png',save_dpi=300)
MTplot(np.transpose(-D2),plot_type='hudson',projection='uv',color=tags,save_file=figpath+'Hudson_aniso_constrainedMT.png',save_dpi=300)

In [ ]:
# compute full time functions and synthetic data by convolving GFs with the time function and weight by MT
pfull_time_func2=np.zeros((ns,6,nt))
psyn_all2 = np.copy(pdxGnx)
for s in range(ns):
    w0 = wavelet(nt=nt, dt=dt, t0=origin_t[s], sig1=sigma1[s], sig2=sigma2[s])
    w = ndimage.shift(w0,-(nt-1)/2.0+0.02/dt) # shift so the first sample corresponds to -20 ms (as the GF's)
    for c in range(6):
        pfull_time_func2[s,c,:]=mt3[s,c]*w
    for itr in range(ntr):
        temp = mt3[s,0]*pGF[s,0,itr,:] + mt3[s,1]*pGF[s,1,itr,:] + mt3[s,2]*pGF[s,2,itr,:] + mt3[s,3]*pGF[s,3,itr,:] + mt3[s,4]*pGF[s,4,itr,:] + mt3[s,5]*pGF[s,5,itr,:]
        psyn_all2[s,itr,:]=dt*np.convolve(temp,w0,mode='same')

In [ ]:
np.savetxt(datapath+"ch6_scaler_MTs_constrained.txt",mt3, fmt='%.9f')
sep.write_file(datapath+"ch6_all_time_func_constrained.H", np.transpose(pfull_time_func2), ds=np.array([dt,1,1]), os=np.array([0,0,0]), dpath=datapath)
sep.write_file(datapath+"ch6_synthetic_constrained.H", np.transpose(psyn_all2), ds=np.array([dt,1,1]), os=np.array([0,-600,0]), dpath=datapath)

In [ ]:
p=0.9
vmax=p*np.amax(pdata[sp])

extent=[-600,600,0.35,0]

fig=plt.figure(figsize=(6.66, 6.66),dpi=300)

ax = plt.subplot(2,2,1)
plt.imshow(np.transpose(pdata[sp]),interpolation='sinc',aspect="auto",extent=extent,cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
# plt.gca().set_yticklabels([])
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'a)',loc='left')

ax = plt.subplot(2,2,2)
plt.imshow(np.transpose(psyn_all[sp]),interpolation='sinc',aspect="auto",extent=extent,cmap='Greys',vmin=-vmax,vmax=vmax)
# plt.xlabel('Offset (m)')
# plt.ylabel('Time (s)')
plt.gca().set_xticklabels([])
plt.gca().set_yticklabels([])
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
#plt.title("DAS data - shot %d" %sp)
# plt.title("DAS observed")
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'b)',loc='left')

ax = plt.subplot(2,2,4)
plt.imshow(np.transpose(psyn_all2[sp]),interpolation='sinc',aspect="auto",extent=extent,cmap='Greys',vmin=-vmax,vmax=vmax)
plt.xlabel('Offset (m)')
plt.ylabel('Time (s)')
# plt.gca().set_yticklabels([])
#plt.colorbar(format='%.'+str(1)+'f',aspect=20)
plt.grid(color='w', linestyle='-', linewidth=0.2)
#plt.title("Modeled data - shot %d" %sp)
# plt.title("DAS modeled")
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.title(r'c)',loc='left')

plt.tight_layout(pad=0., w_pad=-2., h_pad=0.)

plt.savefig(figpath+'das_obs_syn.png',bbox_inches='tight',format='png')